In [2]:
import pandas as pd
import pylab as p
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit, train_test_split
import scipy
import random
import time
from multiprocessing import Pool

In [3]:
core_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CORE.csv')
game_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/GAMES.csv')
team_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/TEAM.csv', index_col=[0])

#win_orig_df = win_orig_df.join(game_df)
offense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/OFFENSE.csv', index_col=[0])
defense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/DEFENSE.csv', index_col=[0])
players_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PLAYERS.csv')
# players_df['player_id'] = players_df.index
# offense_df = offense_df.join(players_df)
# defense_df = defense_df.join(players_df)


In [4]:
print offense_df.columns

Index([u'GID', u'PLAYER', u'PA', u'PC', u'PY', u'INT', u'TDP', u'RA', u'SRA', u'RY', u'TDR', u'TRG', u'REC', u'RECY', u'TDRE', u'FUML', u'PENY', u'FPTS', u'GAME', u'SEAS', u'YEAR', u'TEAM'], dtype='object')


In [5]:
teams = offense_df.groupby('TEAM')

In [6]:
def get_game_features(index, team_game_data, game, players_team):
    feature = {}
    for team_game_row in team_game_data.iterrows():
        this_player_team = False
        # setup which team for this game is this_players team
        if players_team == team_game_row[1]['TNAME']:
            this_player_team = True
        #setup the features from the game data (weather, point spread and stuff)
        for key, val in game.iteritems():
            if key in ignore_columns:
                continue
            if key in categorical_to_binary_columns:
                if val.values[0] != "\N":
                    feature['IS_{0}_{1}_{2}'.format(index, val.values[0], 'my' if this_player_team else 'opp')] = 1.0
            else:
                if str(val.dtype) == "object":
                    if key in convert_to_float_columns:
                        if str.isdigit(val.values[0].strip()):
                            feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)] = float(val.values[0])
                        else:
                            feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)]  = 0.0
                    else:
                        feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)] = val.values[0]
                else:
                    feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)] = val.values[0]
        # setup the features from the teams participating in the game
        for key, val in team_game_row[1].iteritems():
            if key in ignore_columns:
                continue
            if key in categorical_to_binary_columns:
                feature['IS_{0}_{1}_{2}'.format(index, val.values[0], 'my' if this_player_team else 'opp')] = 1.0
            else:
                feature['{0}_{1}_{2}'.format(index, 'my' if this_player_team else 'opp', key)] = val
    return feature

In [7]:
def get_features_for_player(player_id):
    features = []
    offense_player_plays = offense_df[offense_df['PLAYER'] == player_id]
#     defense_player_plays = defense_df[defense_df['PLAYER'] == player_id]
    if j%1000 == 0:
        print "{0}/{1}".format(j, total_count)
    if len(offense_player_plays) > 0:
        offense_player_plays = offense_player_plays.sort(['GID'], ascending=[0])
        for i,row in enumerate(offense_player_plays.iterrows()):
            # target is this games fantasy points
            feature = {'target':row[1]['FPTS']}
            pos = players_df[players_df['PLAYER'] == row[1]['PLAYER']]['POS1'].values[0]
            opp_team = None
            game = game_df[game_df['GID'] == row[1]['GID']]
            if row[1]['TEAM'] == game['V'].values[0]:
                opp_team = game['H'].values[0]
            else:
                opp_team = game['V'].values[0]            
            feature['IS_{0}'.format(pos)] = 1.0
            # features are all the older games not including this one [i:] historical data
            # this means there is a feature for every game for each player
            for k, offesne_player_row in enumerate(offense_player_plays[i+1:(i+5)].iterrows()):
                # this will be the 2 teams that played
                team_game_data = team_df[team_df['GID'] == offesne_player_row[1]['GID']]
                # this is basic info on the game, who is home and visitor
                game = game_df[game_df['GID'] == offesne_player_row[1]['GID']]
                feature.update(get_game_features(k, team_game_data, game, row[1]['TEAM']))
#                 print feature
                # setup the features for the player
                for key, val in offesne_player_row[1].iteritems():
                    if key in ignore_columns:
                        continue
                    if key in categorical_to_binary_columns:
                        feature["IS_{0}_{1}".format(val, k)] = 1.0
                    else:
                        feature["{0}_{1}".format(key, k)] = val
                features.append(feature)
#                 print feature
#                 break
#             break
    return features

In [8]:
ignore_columns = ['PLAYER', 'GID', 'TID', 'TNAME', 'FPTS']
categorical_to_binary_columns = ['V', 'H','STAD', 'WDIR', 'COND', 'SURF', 'TEAM', 'DAY']
convert_to_float_columns = ['HUMD', 'WSPD']
features = []
total_count = len(players_df.index)
for j, row in enumerate(players_df.iterrows()):
    features += get_features_for_player(row[1]['PLAYER'])
#         if j > 10:
#             break
        

0/6541
1000/6541
2000/6541
3000/6541
4000/6541
5000/6541
6000/6541


In [10]:
features_df = pd.DataFrame(features)
features_df = features_df.fillna(0.0)
print features_df
# for i, col in enumerate(features_df.columns):
#     print features_df.columns[i], features_df.dtypes[i]

        0_my_1QP  0_my_2QP  0_my_3QP  0_my_4QP  0_my_BPY  0_my_BRY  0_my_DBP  \
0             14        10         3         6       166       139      14.0   
1             14        10         3         6       166       139      14.0   
2             14        10         3         6       166       139      14.0   
3             14        10         3         6       166       139      14.0   
4              0         0         3        14       135        25      12.5   
5              0         0         3        14       135        25      12.5   
6              0         0         3        14       135        25      12.5   
7              0         0         3        14       135        25      12.5   
8              0         0         3        10       183        71      17.0   
9              0         0         3        10       183        71      17.0   
10             0         0         3        10       183        71      17.0   
11             0         0         3    

In [ ]:
features_df = features_df.fillna(0.0)
if 'target' in features_df.columns:
    targets = features_df['target']
if 'target' in features_df.columns:
    del features_df['target']
x_train, x_test, y_train, y_test = train_test_split(features_df, targets, test_size=0.33, random_state=42)
cfr = ExtraTreesRegressor(
    oob_score = True, bootstrap=True, verbose = 1, max_features = None, n_estimators = 1000, min_samples_leaf = 1,
    n_jobs=7,
    random_state=0,
)
cfr.fit(x_train, y_train)



In [ ]:
score = cfr.score(x_test, y_test)
print score

In [ ]:
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), cfr.feature_importances_), x_train.columns), 
             reverse=True)

In [ ]:
predictions = cfr.predict(x_test)
plt.scatter(y_test, predictions)
plt.show()